In [1]:
import os
os.chdir(os.pardir)

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import matplotlib.patches as mpatches
import pandas as pd

import vapor
from vapor.models import Cambium

2020-10-27 13:45:59,592 [MainThread  ] [INFO ]  Starting log for vapor...


In [2]:
os.getcwd()

'/srv/data/home/skoebric/vapor-open'

In [3]:
# --- Load Regional Best ---
results = pd.read_pickle(os.path.join('results','bnef_results.pkl'))
results['scenario'] = 'Mid Cost'
results['batt_size'] = 0
results.rename({'geometry':'point', 'region':'pca'}, axis='columns', inplace=True)


In [4]:
region_map = pd.read_csv(os.path.join('data','geography','region_hierarchy.csv'))
region_map = region_map[['pca','rto','census_reg','inter']].drop_duplicates()

census_reg_dict = {
    'PA':'West',
    'MTN':'West',
    'ENC':'Midwest',
    'WNC':'Midwest',
    'WSC':'South',
    'MA':'Northeast',
    'NE':'Northeast',
    'ESC':'South',
    'SA':'South'
}
region_map['census_div'] = region_map['census_reg'].map(census_reg_dict)

df = results.merge(region_map, on='pca', how='left')


In [16]:
set(df.columns)

{'6par_Adj',
 '6par_Il',
 '6par_Io',
 '6par_Rs',
 '6par_Rsh',
 '6par_a',
 'ac_loss',
 'adjusted_installed_cost',
 'analysis_period',
 'analysis_period_irr',
 'annual_ac_battery_loss_percent',
 'annual_ac_gross',
 'annual_ac_inv_clip_loss_percent',
 'annual_ac_inv_eff_loss_percent',
 'annual_ac_inv_pnt_loss_percent',
 'annual_ac_inv_pso_loss_percent',
 'annual_ac_inv_tdc_loss_percent',
 'annual_ac_lifetime_loss_percent',
 'annual_ac_loss_ond',
 'annual_ac_perf_adj_loss_percent',
 'annual_ac_wiring_loss',
 'annual_ac_wiring_loss_percent',
 'annual_dc_battery_loss_percent',
 'annual_dc_diodes_loss',
 'annual_dc_diodes_loss_percent',
 'annual_dc_gross',
 'annual_dc_invmppt_loss',
 'annual_dc_lifetime_loss_percent',
 'annual_dc_loss_ond',
 'annual_dc_mismatch_loss',
 'annual_dc_mismatch_loss_percent',
 'annual_dc_module_loss_percent',
 'annual_dc_mppt_clip_loss_percent',
 'annual_dc_nameplate_loss',
 'annual_dc_nameplate_loss_percent',
 'annual_dc_net',
 'annual_dc_nominal',
 'annual_dc_opt

In [24]:
cs = [c for c in df.columns if 'cambium_total_value_2' in c]
t = df[['lifetime_cambium_total_value','cambium_total_value_2020']]
t['sum'] = df[cs].sum(axis=1)
t

,lifetime_cambium_total_value,cambium_total_value_2020,sum
0,3.220295e+06,447.993553,2737.839221
1,3.220295e+06,447.993553,2737.839221
2,2.062747e+07,901.026609,19657.759067
3,1.302743e+07,979.125308,11949.992034
4,1.204821e+08,8392.370908,111896.601673
...,...,...,...
970,4.066222e+07,1261.886995,39271.446845
971,9.318094e+07,3051.448609,88747.615561
972,3.250240e+08,12515.854760,316411.558600
973,3.250240e+08,12515.854760,316411.558600


In [23]:
df[cs].sum(axis=1)

0        2737.839221
1        2737.839221
2       19657.759067
3       11949.992034
4      111896.601673
           ...      
970     39271.446845
971     88747.615561
972    316411.558600
973    316411.558600
974    716972.879934
Length: 975, dtype: float64

In [6]:
years = [str(i) for i in range(2014,2051,1)]
cambium_cols = [i for i in df.columns if 'cambium' in i]
cambium_cols = [i for i in cambium_cols if i[-4:] in years]


# --- subset cambium columns ---
c_df = df[cambium_cols + list(region_map.columns) + ['project_id','tech','system_capacity']]

# --- melt ---
c_df = c_df.melt(id_vars= list(region_map.columns) +['project_id','tech','system_capacity'])
c_df['year'] = [int(i[-4:]) for i in c_df['variable']]
c_df['variable'] = [str(i[:-5]) for i in c_df['variable']]

c_df['value_mw'] = c_df['value'] / c_df['system_capacity']


In [7]:
value_col = 'value'
grouped = c_df.groupby(['census_div','tech','year','variable'], as_index=False)[['system_capacity',value_col]].sum()
grouped['census_div'] = pd.Categorical(grouped['census_div'], categories=['Northeast','West','Midwest','South'], ordered=True)
grouped.sort_values('census_div', inplace=True)

grouped = grouped.loc[grouped['year'] == 2020]
grouped = grouped.loc[grouped['variable'].isin(variable_dict.keys())]

grouped.pivot_table(
    index=['census_div','tech'],
    columns='variable',
    values='value'
)

NameError: name 'variable_dict' is not defined

In [ ]:
df = c_df.copy()
df['census_div'] = pd.Categorical(df['census_div'], categories=['Northeast','West','Midwest','South'], ordered=True)
df.sort_values('census_div', inplace=True)

variable_dict = {
    'cambium_co2_rate_avg':'CO2 Offset',
    'cambium_grid_value':'Grid Value',
#     'cambium_capacity_value':'Capacity Value',
}

df = df.loc[df['variable'].isin(variable_dict.keys())]

g = sns.catplot(x='census_div', y='value_mw',
                hue='tech', col='variable',
               data=df, kind='violin', split=True, bw=.3,
               height=5, aspect=1.2)

In [ ]:
df

In [ ]:
bars.pivot_table(index='census_div', columns='tech', values='value').plot.bar(stacked=True)

In [ ]:
# --- plot ---
g_df = c_df.groupby(['year','variable','tech'], as_index=False)['value'].sum()
g_df = g_df.loc[g_df['variable'] == 'cambium_total_value']
sns.lineplot(x='year', y='value', hue='tech', data=g_df)
